### Лабораторная работа №6

#### <span style="color:blue">Задание.</span> Провести классификацию найденного датасета, методами CatBoost . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

Сначала мы импортируем необходимые библиотеки и нормализуем данные:

In [29]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import cross_val_score


data = pd.read_csv("adult.data")
column_names = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 
                'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 
                'hours-per-week', 'native-country', 'income']
data.columns = column_names

data = data.replace(' ?', pd.NA)

data = data.dropna()
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country'])
data['income'] = data['income'].map({' <=50K': 0, ' >50K': 1})

Разделение данных на обучающий и тестовый наборы

In [30]:
X = data.drop('income', axis=1)
y = data['income']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Метод CatBoost

In [31]:
catboost_model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, loss_function='Logloss', cat_features=[1, 3, 5, 6, 7, 8, 9, 13])

catboost_model.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=20, verbose=10)

y_pred_catboost = catboost_model.predict(X_test)
accuracy_catboost = accuracy_score(y_test, y_pred_catboost)
report_catboost = classification_report(y_test, y_pred_catboost)

print(f'CatBoost Accuracy: {accuracy_catboost}')
print('CatBoost Classification Report:\n', report_catboost)

0:	learn: 0.5892317	test: 0.5910329	best: 0.5910329 (0)	total: 39.4ms	remaining: 3.9s
10:	learn: 0.3385022	test: 0.3428467	best: 0.3428467 (10)	total: 398ms	remaining: 3.22s
20:	learn: 0.3096804	test: 0.3159392	best: 0.3159392 (20)	total: 749ms	remaining: 2.82s
30:	learn: 0.2994047	test: 0.3065486	best: 0.3065486 (30)	total: 1.14s	remaining: 2.54s
40:	learn: 0.2919183	test: 0.2993282	best: 0.2993282 (40)	total: 1.51s	remaining: 2.18s
50:	learn: 0.2876034	test: 0.2958687	best: 0.2958687 (50)	total: 1.89s	remaining: 1.82s
60:	learn: 0.2838437	test: 0.2933552	best: 0.2933552 (60)	total: 2.28s	remaining: 1.46s
70:	learn: 0.2810222	test: 0.2915682	best: 0.2915682 (70)	total: 2.66s	remaining: 1.08s
80:	learn: 0.2785026	test: 0.2901657	best: 0.2901657 (80)	total: 3.04s	remaining: 713ms
90:	learn: 0.2764983	test: 0.2890547	best: 0.2890547 (90)	total: 3.43s	remaining: 339ms
99:	learn: 0.2748264	test: 0.2886354	best: 0.2885855 (98)	total: 3.78s	remaining: 0us

bestTest = 0.2885854755
bestIterati

Выбор гиперпараметров CatBoost:
- iterations=100: Количество итераций (деревьев) для обучения. Увеличение числа итераций может улучшить производительность, но требует больше времени для обучения.
- depth=6: Максимальная глубина дерева. Это помогает избежать переобучения. Мы выбрали относительно большую глубину для более сложной модели.
- learning_rate=0.1: Скорость обучения. Определяет величину коррекции весов на каждой итерации. Мы выбрали стандартное значение 0.1.
- loss_function='Logloss': Функция потерь для бинарной классификации.

In [32]:
cv_scores_catboost = cross_val_score(catboost_model, X, y, cv=5)

print('CatBoost Cross-Validation Scores:', cv_scores_catboost)
print(f'Mean Accuracy (CatBoost): {cv_scores_catboost.mean()}')

0:	learn: 0.5924921	total: 38.6ms	remaining: 3.83s
1:	learn: 0.5281193	total: 70.7ms	remaining: 3.46s
2:	learn: 0.4707459	total: 105ms	remaining: 3.4s
3:	learn: 0.4329761	total: 147ms	remaining: 3.54s
4:	learn: 0.4067748	total: 181ms	remaining: 3.44s
5:	learn: 0.3853846	total: 213ms	remaining: 3.34s
6:	learn: 0.3726855	total: 248ms	remaining: 3.3s
7:	learn: 0.3614815	total: 286ms	remaining: 3.29s
8:	learn: 0.3511060	total: 322ms	remaining: 3.25s
9:	learn: 0.3433445	total: 362ms	remaining: 3.26s
10:	learn: 0.3379253	total: 392ms	remaining: 3.17s
11:	learn: 0.3341878	total: 427ms	remaining: 3.13s
12:	learn: 0.3298984	total: 464ms	remaining: 3.11s
13:	learn: 0.3269076	total: 498ms	remaining: 3.06s
14:	learn: 0.3236768	total: 535ms	remaining: 3.03s
15:	learn: 0.3203760	total: 576ms	remaining: 3.02s
16:	learn: 0.3185994	total: 613ms	remaining: 2.99s
17:	learn: 0.3161556	total: 650ms	remaining: 2.96s
18:	learn: 0.3142776	total: 696ms	remaining: 2.96s
19:	learn: 0.3118706	total: 736ms	remaini

64:	learn: 0.2821948	total: 2.58s	remaining: 1.39s
65:	learn: 0.2821432	total: 2.6s	remaining: 1.34s
66:	learn: 0.2819493	total: 2.64s	remaining: 1.3s
67:	learn: 0.2817283	total: 2.67s	remaining: 1.26s
68:	learn: 0.2815087	total: 2.7s	remaining: 1.21s
69:	learn: 0.2813211	total: 2.74s	remaining: 1.17s
70:	learn: 0.2811767	total: 2.78s	remaining: 1.13s
71:	learn: 0.2807195	total: 2.82s	remaining: 1.09s
72:	learn: 0.2804952	total: 2.85s	remaining: 1.06s
73:	learn: 0.2801507	total: 2.89s	remaining: 1.01s
74:	learn: 0.2797746	total: 2.92s	remaining: 974ms
75:	learn: 0.2796558	total: 2.96s	remaining: 935ms
76:	learn: 0.2793947	total: 3s	remaining: 895ms
77:	learn: 0.2791746	total: 3.03s	remaining: 855ms
78:	learn: 0.2790359	total: 3.08s	remaining: 818ms
79:	learn: 0.2787331	total: 3.12s	remaining: 779ms
80:	learn: 0.2785665	total: 3.15s	remaining: 739ms
81:	learn: 0.2784384	total: 3.19s	remaining: 701ms
82:	learn: 0.2781112	total: 3.23s	remaining: 663ms
83:	learn: 0.2777855	total: 3.28s	rem

26:	learn: 0.3049029	total: 916ms	remaining: 2.48s
27:	learn: 0.3043192	total: 953ms	remaining: 2.45s
28:	learn: 0.3034528	total: 994ms	remaining: 2.43s
29:	learn: 0.3027564	total: 1.03s	remaining: 2.41s
30:	learn: 0.3020379	total: 1.07s	remaining: 2.38s
31:	learn: 0.3009735	total: 1.11s	remaining: 2.35s
32:	learn: 0.3002896	total: 1.15s	remaining: 2.33s
33:	learn: 0.2991712	total: 1.18s	remaining: 2.3s
34:	learn: 0.2986635	total: 1.22s	remaining: 2.27s
35:	learn: 0.2981520	total: 1.26s	remaining: 2.23s
36:	learn: 0.2971033	total: 1.3s	remaining: 2.21s
37:	learn: 0.2963415	total: 1.33s	remaining: 2.17s
38:	learn: 0.2954661	total: 1.37s	remaining: 2.14s
39:	learn: 0.2950107	total: 1.41s	remaining: 2.11s
40:	learn: 0.2945704	total: 1.45s	remaining: 2.08s
41:	learn: 0.2939622	total: 1.48s	remaining: 2.05s
42:	learn: 0.2935090	total: 1.52s	remaining: 2.02s
43:	learn: 0.2931513	total: 1.56s	remaining: 1.98s
44:	learn: 0.2926094	total: 1.59s	remaining: 1.95s
45:	learn: 0.2921289	total: 1.63s

92:	learn: 0.2781268	total: 3.9s	remaining: 294ms
93:	learn: 0.2779814	total: 3.94s	remaining: 252ms
94:	learn: 0.2778165	total: 3.99s	remaining: 210ms
95:	learn: 0.2776591	total: 4.02s	remaining: 168ms
96:	learn: 0.2773498	total: 4.06s	remaining: 126ms
97:	learn: 0.2771211	total: 4.1s	remaining: 83.7ms
98:	learn: 0.2769002	total: 4.14s	remaining: 41.8ms
99:	learn: 0.2767624	total: 4.17s	remaining: 0us
CatBoost Cross-Validation Scores: [0.86490966 0.8645557  0.87168435 0.86986074 0.87102122]
Mean Accuracy (CatBoost): 0.8684063358864883
